In [4]:
!pip install pyclustertend

In [5]:
%pylab inline
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from sklearn import preprocessing
from collections import defaultdict
import pyclustertend
from dfply import *
from scipy.cluster.hierarchy import dendrogram

Populating the interactive namespace from numpy and matplotlib


In [6]:
df = pd.read_csv('./data/city_temperature.csv')

/Users/inzamamrahaman/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.head(100)

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9
...,...,...,...,...,...,...,...,...
95,Africa,Algeria,NaN,Algiers,4,6,1995,59.0
96,Africa,Algeria,NaN,Algiers,4,7,1995,54.9
97,Africa,Algeria,NaN,Algiers,4,8,1995,54.2
98,Africa,Algeria,NaN,Algiers,4,9,1995,57.8


In [8]:
df.drop(['Region', 'Country', 'State'], axis=1, inplace=True)

In [10]:
df.head()

,City,Month,Day,Year,AvgTemperature
0,Algiers,1,1,1995,64.2
1,Algiers,1,2,1995,49.4
2,Algiers,1,3,1995,48.8
3,Algiers,1,4,1995,46.4
4,Algiers,1,5,1995,47.9


In [11]:
df = df.groupby(['City', 'Month']).agg({'AvgTemperature': 'mean'}).reset_index()

In [12]:
df.head(100)

,City,Month,AvgTemperature
0,Abidjan,1,74.431266
1,Abidjan,2,75.821088
2,Abidjan,3,79.926427
3,Abidjan,4,77.242564
4,Abidjan,5,78.142640
...,...,...,...
95,Algiers,12,52.410052
96,Allentown,1,29.665509
97,Allentown,2,32.243810
98,Allentown,3,40.130025


In [16]:
df1 = df >> group_by(X.City) >> summarise(
    Jan=X.AvgTemperature[X.Month == 1].mean(),
    Feb=X.AvgTemperature[X.Month == 2].mean(),
    Mar=X.AvgTemperature[X.Month == 3].mean(),
    Apr=X.AvgTemperature[X.Month == 4].mean(),
    May=X.AvgTemperature[X.Month == 5].mean(),
    Jun=X.AvgTemperature[X.Month == 6].mean(),
    Jul=X.AvgTemperature[X.Month == 7].mean(),
    Aug=X.AvgTemperature[X.Month == 8].mean(),
    Sept=X.AvgTemperature[X.Month == 9].mean(),
    Oct=X.AvgTemperature[X.Month == 10].mean(),
    Nov=X.AvgTemperature[X.Month == 11].mean(),
    Dec=X.AvgTemperature[X.Month == 12].mean()
)

In [17]:
df1.head()

,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
0,Abidjan,74.431266,75.821088,79.926427,77.242564,78.142640,72.168933,70.482839,73.145161,73.064000,79.626323,73.298667,73.917010
1,Abilene,44.661042,48.935374,56.533127,64.633333,72.460279,77.648400,83.410710,82.541161,75.680933,65.830710,54.590133,44.587887
2,Abu Dhabi,66.038089,69.240408,74.353598,80.914359,88.488325,92.023600,96.446581,96.470581,91.776533,85.837935,76.617867,68.759794
3,Addis Ababa,18.053722,30.645306,27.794293,19.662436,20.103939,22.750667,20.884774,29.490323,32.118800,30.286581,35.161867,19.769231
4,Akron Canton,26.981390,29.519592,37.888586,49.582821,60.356853,68.041467,72.571613,70.736774,64.121733,52.941032,41.420400,31.454194


In [18]:
X = df1.loc[:,'Jan':]

In [19]:
y = df1['City']

In [20]:
X

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Nov,Dec
0,74.431266,75.821088,79.926427,77.242564,78.142640,72.168933,70.482839,73.145161,73.064000,79.626323,73.298667,73.917010
1,44.661042,48.935374,56.533127,64.633333,72.460279,77.648400,83.410710,82.541161,75.680933,65.830710,54.590133,44.587887
2,66.038089,69.240408,74.353598,80.914359,88.488325,92.023600,96.446581,96.470581,91.776533,85.837935,76.617867,68.759794
3,18.053722,30.645306,27.794293,19.662436,20.103939,22.750667,20.884774,29.490323,32.118800,30.286581,35.161867,19.769231
4,26.981390,29.519592,37.888586,49.582821,60.356853,68.041467,72.571613,70.736774,64.121733,52.941032,41.420400,31.454194
...,...,...,...,...,...,...,...,...,...,...,...,...
316,27.928674,31.765354,43.667025,55.220370,62.188530,69.876296,80.337276,80.356631,63.828889,51.512545,41.710833,30.670161
317,26.846526,29.116463,37.119727,48.742949,59.311421,66.679733,71.259097,69.071226,62.768800,52.227226,41.256267,31.382710
318,52.066398,52.397345,57.986828,65.864167,66.308333,74.141111,78.307527,79.781183,76.585000,64.306183,55.921944,44.327913
319,23.587965,32.703946,39.463772,48.792949,56.154003,62.488933,65.772258,64.244645,55.302933,48.045161,38.364800,29.084021


In [21]:
model = cluster.AgglomerativeClustering(distance_threshold=0, n_clusters=None)

In [22]:
model.fit(X)

AgglomerativeClustering(distance_threshold=0, n_clusters=None)

In [23]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [24]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(100, 100)

<Figure size 7200x7200 with 0 Axes>

In [25]:
ax = fig.add_subplot(1, 1, 1)

In [26]:

plot_dendrogram(model, labels=list(y), ax=ax)

In [27]:
x = plt.setp(ax.get_xticklabels(), fontsize=25)

In [31]:
fig.savefig('/Users/inzamamrahaman/Desktop/non_guardian/teaching/COMP6940-2023/Labs/dsProject/7/temp.png')

In [29]:
!ls

Categorical NB.ipynb
Classification Example.ipynb
Classification.ipynb
CornacExp-2023-03-22_14-47-12-647981.log
CornacExp-2023-03-22_14-51-46-993467.log
Example2.ipynb
Gaussian NB.ipynb
HClustering Example.ipynb
KMeans.ipynb
Ratings.ipynb
Scientific Rule mining.ipynb
Simple Example.ipynb
Simple Rule Mining.ipynb
Untitled.ipynb
Untitled1.ipynb
data
kNN.ipynb
recommendation_system.ipynb
temp.png


In [30]:
!pwd

/Users/inzamamrahaman/Desktop/non_guardian/teaching/COMP6940-2023/Labs/dsProject/7
